In [ ]:
pip install nltk

In [1]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import re
import nltk
from nltk.util import ngrams
import itertools as it
import pandas as pd
from collections import Counter
import numpy as np

In [3]:
learning = open('Learning', 'r')  # learning dataset

testing = open('Testing', 'r')  # testing dataset
# ------------Preparing text----------------------
raw_text_learn = learning.read().lower()
text_learn = raw_text_learn.replace("-", " ")
text_cleaned_learn = re.sub(r'[^\w\s]', '', text_learn)
learn_split = text_cleaned_learn.split()  # text splitted on words

unique_learn = list(set(learn_split))  # list of unique words

raw_text_test = testing.read().lower()
text_test = raw_text_test.replace("-", " ")
text_cleaned_test = re.sub(r'[^\w\s]', '', text_test)
test_split = text_cleaned_test.split()

unique_test = list(set(test_split))

lyambda1 = 0.2
lyambda2 = 0.7
# ----------------------Unigrams on learning text----------------------------
basic_freq, basic_words = [], []

for word in unique_learn:
    word_count = 0
    basic_words.append(word)
    for word2 in learn_split:
        if (word == word2):
            word_count += 1
    basic_freq.append(word_count / len(learn_split))

uniDF = pd.DataFrame(data={'Words': unique_learn, 'Probabilities': basic_freq})
# print(uniDF)

n = len(learn_split)
v = len(unique_learn)
test = Counter(test_split)

# ---------------------Unigrams smoothing on testing text---------------------
uni_words, lapl_freq, lmbd1_freq, lmbd2_freq = [], [], [], []
for word, freq in zip(test.keys(), test.values()):
    uni_words.append(word)
    if word in learn_split:
        lapl_freq.append((freq + 1) / (n + v))
        lmbd1_freq.append((freq + lyambda1) / (n + v * lyambda1))
        lmbd2_freq.append((freq + lyambda2) / (n + v * lyambda2))

    else:
        lapl_freq.append(1 / (n + v))
        lmbd1_freq.append(lyambda1 / (n + v * lyambda1))
        lmbd2_freq.append(lyambda2 / (n + v * lyambda2))

laplDF = pd.DataFrame(data={'Words': uni_words, 'Probabilities': lapl_freq})
print(laplDF)
print()

lmbd1DF = pd.DataFrame(data={'Words': uni_words, 'Probabilities': lmbd1_freq})
print(lmbd1DF)
print()

lmbd2DF = pd.DataFrame(data={'Words': uni_words, 'Probabilities': lmbd2_freq})
print(lmbd2DF)
print()

# ----------------------------Bigrams on learning text---------------------------
learn_bigrams = list(ngrams(learn_split, 2))
learn_unigrams = list(ngrams(learn_split, 1))

bi_basic_freq = []

bigrams = Counter(learn_bigrams)
unigrams = Counter(learn_unigrams)

for bigram in bigrams:
    for unigram in unigrams:
        if (bigram[0] == unigram[0]):
            bi_basic_freq.append(bigrams[bigram] / unigrams[unigram])

biDF = pd.DataFrame(data={'Words': list(bigrams.keys()), 'Probabilities': bi_basic_freq})
print(biDF)
print()
# ----------------------Bigrams smooting on testing text---------------------------
b = len(learn_bigrams)

test_bigrams = list(ngrams(test_split, 2))
test_unigrams = list(ngrams(test_split, 1))

bigrams_learn = Counter(learn_bigrams)
unigrams_learn = Counter(learn_unigrams)
bigrams_test = Counter(test_bigrams)
unigrams_test = Counter(test_unigrams)

bi_words, bi_lapl_freq, bi_lmbd1_freq, bi_lmbd2_freq = [], [], [], []
for bigram, freq in zip(bigrams_test.keys(), bigrams_test.values()):
    for unigram in unigrams_test.keys():
        if bigram[0] == unigram[0]:
            bi_words.append(bigram)
            if bigram in bigrams_learn:
                bi_lapl_freq.append((bigrams_test[bigram] + 1) / (unigrams_test[unigram] + v ** 2))
                bi_lmbd1_freq.append((bigrams_test[bigram] + lyambda1) / (unigrams_test[unigram] + b * lyambda1))
                bi_lmbd2_freq.append((bigrams_test[bigram] + lyambda2) / (unigrams_test[unigram] + b * lyambda2))

            else:
                bi_lapl_freq.append(1 / (v ** 2))
                bi_lmbd1_freq.append(lyambda1 / (b * lyambda1))
                bi_lmbd2_freq.append(lyambda2 / (b * lyambda2))

bi_laplDF = pd.DataFrame(data={'Words': bi_words, 'Probabilities': bi_lapl_freq})
print(bi_laplDF)
print()

bi_lmbd1DF = pd.DataFrame(data={'Words': bi_words, 'Probabilities': bi_lmbd1_freq})
print(bi_lmbd1DF)
print()

bi_lmbd2DF = pd.DataFrame(data={'Words': bi_words, 'Probabilities': bi_lmbd2_freq})
print(bi_lmbd2DF)
print()
# --------------------------Perplexity---------------------------------------------

lapl_perp = (1 / np.prod(laplDF['Probabilities'])) ** (1 / len(test_split))
lmbd1_perp = (1 / np.prod(lmbd1DF['Probabilities'])) ** (1 / len(test_split))
lmbd2_perp = (1 / np.prod(lmbd2DF['Probabilities'])) ** (1 / len(test_split))

print("Unigrams Perplexity:")
print("Laplasian - ", lapl_perp)
print("Lyambda1 - ", lmbd1_perp)
print("Lyambda2 - ", lmbd2_perp)

bi_lapl_perp = (1 / np.prod(bi_laplDF['Probabilities'])) ** (1 / len(test_bigrams))
bi_lmbd1_perp = (1 / np.prod(bi_lmbd1DF['Probabilities'])) ** (1 / len(test_bigrams))
bi_lmbd2_perp = (1 / np.prod(bi_lmbd2DF['Probabilities'])) ** (1 / len(test_bigrams))

print()
print("Bigrams Perplexity:")
print("Laplasian - ", bi_lapl_perp)
print("Lyambda1 - ", bi_lmbd1_perp)
print("Lyambda2 - ", bi_lmbd2_perp)

       Words  Probabilities
0        вот       0.035088
1        кот       0.017544
2    который       0.052632
3     пугает       0.017544
4          и       0.017544
5      ловит       0.017544
6     синицу       0.017544
7    которая       0.052632
8      часто       0.035088
9     ворует       0.035088
10   пшеницу       0.035088
11         в       0.052632
12    тёмном       0.035088
13    чулане       0.035088
14  хранится       0.035088
15      доме       0.035088
16  построил       0.035088
17      джек       0.035088

       Words  Probabilities
0        вот       0.029268
1        кот       0.004878
2    который       0.053659
3     пугает       0.004878
4          и       0.004878
5      ловит       0.004878
6     синицу       0.004878
7    которая       0.053659
8      часто       0.029268
9     ворует       0.029268
10   пшеницу       0.029268
11         в       0.053659
12    тёмном       0.029268
13    чулане       0.029268
14  хранится       0.029268
15      доме       